In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm #progress bar

#For data extracion
import requests
import json
import os

# NLP
import spacy #spacy for quick language prepro
nlp = spacy.load('en_core_web_sm') #instantiating English moduleb


## Topic modeling imports
from gensim.corpora.dictionary import Dictionary # Import the dictionary builder
from gensim.models import LdaMulticore # we'll use the faster multicore version of LDA
import pyLDAvis # viz
import pyLDAvis.gensim_models as gensimvis

%matplotlib inline
pyLDAvis.enable_notebook()

import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions

In [4]:
from chromadb.config import Settings
client = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory="../chroma_with_Hamids_help/" # Optional, defaults to .chromadb/ in the current directory
))

In [5]:
paper_sentences_col = client.get_collection("sentence_emb_1")

No embedding_function provided, using default embedding function: DefaultEmbeddingFunction https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2


In [6]:
paper_sentences_col.get()

{'ids': ['YH7II5_280',
  'JHILCV_131',
  'B7LZB7_2009',
  'B7LZB7_2010',
  'UDILNI_4',
  'UDILNI_5',
  'UDILNI_6',
  'UDILNI_7',
  'UDILNI_8',
  'UDILNI_9',
  'UDILNI_10',
  'UDILNI_11',
  'UDILNI_12',
  'UDILNI_13',
  'UDILNI_14',
  'UDILNI_15',
  'UDILNI_16',
  'UDILNI_17',
  'UDILNI_18',
  'UDILNI_19',
  'UDILNI_20',
  'UDILNI_21',
  'UDILNI_22',
  'UDILNI_23',
  'UDILNI_24',
  'UDILNI_25',
  'UDILNI_26',
  'UDILNI_27',
  'UDILNI_28',
  'UDILNI_29',
  'UDILNI_30',
  'UDILNI_31',
  'UDILNI_32',
  'UDILNI_33',
  'UDILNI_34',
  'UDILNI_35',
  'UDILNI_36',
  'UDILNI_37',
  'UDILNI_38',
  'UDILNI_39',
  'UDILNI_40',
  'UDILNI_41',
  'UDILNI_42',
  'UDILNI_43',
  'UDILNI_44',
  'UDILNI_45',
  'UDILNI_46',
  'UDILNI_47',
  'UDILNI_48',
  'UDILNI_49',
  'UDILNI_50',
  'UDILNI_51',
  'UDILNI_52',
  'UDILNI_53',
  'UDILNI_54',
  'UDILNI_55',
  'UDILNI_56',
  'UDILNI_57',
  'UDILNI_58',
  'UDILNI_59',
  'UDILNI_60',
  'UDILNI_61',
  'UDILNI_62',
  'UDILNI_63',
  'UDILNI_64',
  'UDILNI_65',
  '

Here we need a pull from from the mongo collection and the chroma collection to try it out on 5 documents. 

In [8]:
results = paper_sentences_col.query(
    query_texts="impact of globalization",
    n_results=10
)


In [9]:
results["documents"][0]

['Does Globalization Affect Growth',
 'Does Globalization Affect Growth',
 ' The effects of globalization on growth have also been frequently analyzed with these measures',
 ' (2000) Globalization: Myths, Facts and Consequences ',
 '  The next section analyzes the influence of these dimensions of globalization on  economic growth',
 '   9   While the overall effect of globalization  on growth was found to be positiv e, it is  interesting to examine the effects of the single components',
 '  Beer, Linda; Terry Boswell (2001), The Effect s of Globalization on Inequality: A Cross- National Analysis, Halle Institut Occasional Paper',
 ' Introduction  Many non-economists expect the costs associated with globalization to exceed its benefits',
 ' While those studies provide very detailed analysis of individual sub-dimensions of globalization, none of them examines the consequences of globalization on economic growth in greater detail',
 ' It is not obvious that economic,  cultural and politic

In [10]:
tokens = []
pbar2 = tqdm(len(results['documents'][0]),position=0,leave=True)

for abstract in nlp.pipe(results['documents'][0], disable=["ner"]):
  proj_tok = [token.lemma_.lower() for token in abstract 
              if token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV'] 
              and not token.is_stop
              and not token.is_punct] 
  tokens.append(proj_tok)
  pbar2.update(1)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


0it [00:00, ?it/s]

In [68]:
texts['tokens'] = tokens
texts

,Unnamed: 0,text,summary,tokens
0,0,attention need \n ashish vaswani \n google ...,"This paper proposes the Transformer, a model ...","[attention, ashish, google, brain, avaswani@go..."
1,1,beneﬁts biophysical synapses \n julian lemme...,This paper examines the use of biophysical sy...,"[biophysical, synapsis, julian, lemmel, radu, ..."
2,2,morea gpu accelerated evolutionary algorithm...,"This paper presents MOREA, a GPU-accelerated ...","[morea, gpu, evolutionary, algorithm, objectiv..."
3,3,performance indicators use self adaptation \...,This paper investigates the use of self-adapt...,"[performance, indicator, self, adaptation, mul..."
4,4,affine combinations bbob problems performanc...,This paper examines the use of affine combina...,"[affine, combination, bbob, problem, performan..."
5,5,arxiv:2303.04347v1 cs ne 8 mar 2023publi...,This paper presents a method for converting a...,"[arxiv:2303.04347v1, cs, ne, mar, conference, ..."
6,6,evolutionary reinforcement learning survey \...,This article provides an overview of Evolutio...,"[evolutionary, reinforcement, learning, survey..."
7,7,radam t exture recognition randomized \n agg...,This paper proposes a new method called Rando...,"[radam, t, exture, recognition, deepactivation..."
8,8,computer vision enabled damage detection mod...,This paper presents a novel deep learning-bas...,"[computer, vision, damage, detection, model, y..."
9,9,patch invisibility \n naturalistic black box...,This paper proposes a gradient-free method to...,"[invisibility, naturalistic, black, box, adver..."


47it [00:28, 21.90it/s]

In [73]:
# Create dictionary
dictionary_indu = Dictionary(texts['tokens'])

# construct corpus using this dictionary
corpus_indu = [dictionary_indu.doc2bow(doc) for doc in texts['tokens']]

In [74]:
corpus_indu

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 2),
  (8, 2),
  (9, 1),
  (10, 6),
  (11, 1),
  (12, 1),
  (13, 3),
  (14, 1),
  (15, 4),
  (16, 1),
  (17, 3),
  (18, 1),
  (19, 2),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 2),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 2),
  (31, 1),
  (32, 3),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 2),
  (38, 1),
  (39, 3),
  (40, 2),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 2),
  (51, 8),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 2),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 2),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 2),
  (71, 1),
  (72, 3),
  (73, 1),
  (74, 2),
  (75, 1),
  (76, 2),
  (77, 1),
  (78, 12),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 4),
  (83, 4),
  (84, 2),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 1),
  (89, 1),
  (90, 1),
  (91, 1

In [75]:
# Training the model
lda_model_indu = LdaMulticore(corpus_indu, id2word=dictionary_indu, num_topics=10, workers = 4, passes=10)

In [76]:

# Creating the Visualization
lda_display_indu = pyLDAvis.gensim_models.prepare(lda_model_indu, corpus_indu, dictionary_indu)

In [77]:
# Display viz
pyLDAvis.display(lda_display_indu)